#Deploy and Run AI Foundry Models in Azure Databricks
##Install the Azure-AI-Inference SDK

🔍 Use Case: AI Assistant for Company Policies
🎯 Goal:
Build an AI assistant that answers employee questions about company policies (e.g., leave policy, work from home rules) by:

Searching documents stored in Databricks (PDFs, Word files, etc.)

Using Azure OpenAI (via azure-ai-inference) to generate the final answer

⚙️ Steps Overview:
Ingest documents into a Delta table

Split text into chunks + store embeddings

Enable vector search using databricks-vectorsearch

Retrieve top relevant chunks

Use azure-ai-inference to generate the final answer

In [0]:
%pip install azure-ai-inference
     

In [0]:
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.ai.inference.models import ImageContentItem, ImageUrl, TextContentItem
from azure.core.credentials import AzureKeyCredential

##Populate the Environment Variables

In [0]:
ai_foundry_model_endpoint = ""
ai_foundry_model_api_key = ""
ai_foundry_model_deployment_name = ""


##Optional!1

In [0]:
with open("/dbfs/FileStore/tables/leave_policy.txt", "r") as f:
    policy_text = f.read()

user_prompt = f"""Context: {policy_text}

Question: {your_input_question}"""


##Optional2

In [0]:
try:
    response = call_ai_foundry_catalog_model(...)
    print("AI Response:", response)
except Exception as e:
    print("Error calling model:", e)

##Create a function to call the AI Foundry Model

In [0]:
def call_ai_foundry_catalog_model(user_prompt, system_prompt, ai_foundry_model_endpoint, ai_foundry_model_api_key):
    client = ChatCompletionsClient(
        endpoint = ai_foundry_model_endpoint,
        credential = AzureKeyCredential(ai_foundry_model_api_key),
        api_version = "2024-05-01-preview"
    )

    messages = [
        {
            "role":"system",
            "content":f"{system_prompt}"
        },
        {
            "role":"user",
            "content":[
                {
                    "type":"text",
                    "text":f"{user_prompt}"
                }
            ]
        }
    ]

    response = client.complete(
        messages=messages,
        temperature=0.7,
        
    )

    return response.choices[0].message.content


##Create a Basic Chat App

In [0]:

system_prompt = "You are a helpful assistant."
while True:
    user_prompt = input("Enter your message: ")
    response = call_ai_foundry_catalog_model(user_prompt, system_prompt, ai_foundry_model_endpoint, ai_foundry_model_api_key)
    print("AI Response: ", response)
    if user_prompt == "exit":
        break

##Futuyreupligfft 

##Deploy with a Simple Web UI (Best for Internal Use)

In [0]:
import gradio as gr

def chat_with_ai(user_input):
    return call_ai_foundry_catalog_model(user_input, system_prompt, ai_foundry_model_endpoint, ai_foundry_model_api_key)

gr.ChatInterface(fn=chat_with_ai, title="Company Policy Assistant").launch()


In [0]:
import smtplib
from email.mime.text import MIMEText

msg = MIMEText(ai_response)
msg['Subject'] = "AI Answer: Your HR Policy Question"
msg['From'] = "bot@company.com"
msg['To'] = "user@example.com"

with smtplib.SMTP("smtp.office365.com", 587) as server:
    server.starttls()
    server.login("your_email", "your_password")
    server.send_message(msg)
